In [ ]:
using Gmsh
using GridapGmsh
using Gridap
using Gridap.Geometry
using Gridap.TensorValues
using Gridap.Fields
using Gridap.CellData
using Gridap.ReferenceFEs
using Plots; pyplot()
using LinearAlgebra

In [ ]:
E_mortar = 30
E_brick = 1.93e3
ν_mortar = 0.4
ν_brick = 0.33
@show λ_mortar = E_mortar*ν_mortar/((1+ν_mortar)*(1-2*ν_mortar))
@show λ_brick = E_brick*ν_brick/((1+ν_brick)*(1-2*ν_brick))
@show μ_mortar = E_mortar/(2*(1+ν_mortar))
@show μ_brick = E_brick/(2*(1+ν_brick))
ls = 1.0
Gc_mortar = 0.28 
Gc_brick = 0.73  
const η = 1e-10

In [ ]:
model = GmshDiscreteModel("model.msh")
writevtk(model,"Interlocking_model") 

In [ ]:
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension);
const brick = get_tag_from_name(labels,"brick")
const mortar = get_tag_from_name(labels,"mortar")

In [ ]:
function Gc(s_id,tag)
        if tag == brick          
            return  Gc_brick *s_id
        elseif tag == mortar        
            return Gc_mortar *s_id
        end
end

function material_tag(tag)
    if tag == brick
        μ = μ_brick
        λ = λ_brick
        k = μ+λ
    elseif tag == mortar
        μ = μ_mortar
        λ = λ_mortar
        k = μ+λ
    end 
    return μ,λ,k
end

In [ ]:
function σ_elas(ε,tag)
    μ,λ,k= material_tag(tag)
     σ_elas = λ*tr(ε)*one(ε) + 2*μ*ε
    return  σ_elas
end

function σ_fun(ε, ε_in, s_in,tag)
     
     σ = (s_in^2 + η)*σ_elas(ε,tag)
     
    return σ
end

In [ ]:
function ΨPos(ε_in,tag)
    μ,λ,k= material_tag(tag)
    εArray = get_array(ε_in)
    Λ, P = eigen(εArray)
    ε1 = Λ[1]
    ε2 = Λ[2]
    if ε1 >= 0 &&  ε2 >= 0
        Λpos = [ε1 0; 0 ε2]
        Λneg = [0 0; 0 0]
    elseif ε1 >= 0 &&  ε2 < 0
        Λpos = [ε1 0; 0 0]
        Λneg = [0 0; 0 ε2]
    elseif ε1 < 0 &&  ε2 >= 0
        Λpos = [0 0; 0 ε2]
        Λneg = [ε1 0; 0 0]
    elseif ε1 < 0 &&  ε2 < 0
        Λpos = [0 0; 0 0]
        Λneg = [ε1 0; 0 ε2]
    end 
   εPos = P*Λpos*P'
   εNeg = P*Λneg*P'
   ε_Pos = TensorValue(εPos)
   if tr(ε_in) >= 0
        ΨPlus = 0.5*λ*(tr(ε_in))^2 + μ*(ε_Pos ⊙ ε_Pos) 
   elseif tr(ε_in) < 0
        ΨPlus = μ*(ε_Pos ⊙ ε_Pos)
   end
    return ΨPlus
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["loadboundary","fixedboundary"],
          dirichlet_masks=[(false,true), (true,true)])
uh = zero(V0_Disp)

In [ ]:
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
LoadTagId = get_tag_from_name(labels,"loadboundary")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫(u*v)*dΩ
  b(v) = ∫(v*q)*dΩ
  op = AffineFEOperator(a,b,V,V)
  qh = solve(op)
  return qh
end

sId = CellState(1.0,dΩ)
shId = project(sId,model,dΩ,order)

In [ ]:
function  stepDisp(uh_in,sh_in,vApp,tag)
    uApp1(x) = VectorValue(0.0,vApp)
    uApp2(x) = VectorValue(0.0,0.0)
    U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])
    a_Disp(u,v) = ∫( (ε(v)⊙(σ_fun ∘ (ε(u),ε(uh_in),sh_in,tag)) ) )*dΩ
     b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = Gridap.solve(op_Disp)
    return  uh_out
end

In [ ]:
function   stepPhaseField(uh_in ,ΨPlusPrev_in)
    a_PF(s,ϕ) =∫( (Gc∘(shId,mat_tags))*ls*∇(ϕ)⋅∇(s)+ 2*ΨPlusPrev_in*s*ϕ+ ((Gc∘(shId,mat_tags))/ls)*s*ϕ)*dΩ
    b_PF(ϕ) =∫( ((Gc∘(shId,mat_tags))/ls)*ϕ)*dΩ
    op_PF = AffineFEOperator(a_PF,b_PF ,U_PF ,V0_PF)
    sh_out = Gridap.solve(op_PF)
    return  sh_out
end

In [ ]:
function  new_EnergyState(ΨPlusPrev_in,ΨhPos_in)
    ΨPlus_in = ΨhPos_in
    if ΨPlus_in  >=ΨPlusPrev_in
        ΨPlus_out =ΨPlus_in
    else
        ΨPlus_out=ΨPlusPrev_in
    end
    true,ΨPlus_out
end 

In [ ]:
vApp = 0
const  vAppMax = 5
delv = vAppMax/100
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
Stress = Float64[]
Strain = Float64[]

push!(Load, 0.0)
push!(Displacement, 0.0)

sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
ΨPlusPrev = CellState(0.0,dΩ)

while  vApp .< vAppMax
    count = count  .+ 1   
    vApp = vApp .+ delv
    print("\n Entering  displacemtent  step:count ", float(vApp))

    for  inner = 1: innerMax
        ΨhPlusPrev = project(ΨPlusPrev ,model ,dΩ,order)
        RelErr = abs(sum(∫( (Gc∘(shId,mat_tags))*ls*∇(sh)⋅∇(sh) + 2*ΨhPlusPrev*sh*sh + ((Gc∘(shId,mat_tags))/ls)*sh*sh)*dΩ-∫( ((Gc∘(shId,mat_tags))/ls)*sh)*dΩ))/abs(sum(∫( ((Gc∘(shId,mat_tags))/ls)*sh)*dΩ))
        print("\n Relative Error :", float(RelErr))
        sh = stepPhaseField(uh,ΨhPlusPrev)
        uh = stepDisp(uh,sh,vApp,mat_tags)
        ΨhPos_in = ΨPos∘(ε(uh),mat_tags)
        update_state!( new_EnergyState ,ΨPlusPrev ,ΨhPos_in)
        if   RelErr  < 1e-8
            break
        end
    end
    Node_Force = sum(∫(n_Γ_Load⋅(σ_fun∘(ε(uh),ε(uh),sh,mat_tags)))*dΓ_Load)
    push!(Load , Node_Force[2])
    push!( Displacement , vApp)
    if mod(count,5)==0
    writevtk(Ω,"results_vtk",cellfields= ["uh"=>uh ,"s"=>sh,"epsi"=>ε(uh), "σ_elas"=>σ_elas∘(ε(uh),mat_tags)])
    end
 end

In [ ]:
plot(Displacement,Load)